# **Tradução de Texto - Translator**

In [ ]:
!pip install requests python-docx python-dotenv

In [ ]:
from google.colab import files

# Fazer upload do arquivo .env com variáveis de ambiente do Translator
uploaded = files.upload()

In [ ]:
from dotenv import load_dotenv
from docx import Document
import requests
import os

# Carregar as variáveis do arquivo .env
load_dotenv('translator.env')

# Acessar as variáveis
subscription_key = os.getenv('AZURE_TRANSLATOR_KEY')
endpoint = os.getenv('AZURE_TRANSLATOR_ENDPOINT')
location = os.getenv('AZURE_TRANSLATOR_LOCATION')

# Definição do idioma de tradução
language_destination = 'pt-br'

# Função para tradução de textos
def translator_text(text, target_language):
    path = '/translate'
    constructed_url = endpoint + path
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(os.urandom(16))
    }

    body = [{'text': text}]
    params = {'api-version': '3.0', 'from': 'en', 'to': [target_language]}

    request = requests.post(constructed_url, headers=headers, params=params, json=body)
    response = request.json()
    return response[0]['translations'][0]['text']

# Chamada para a função passando os parâmetros do texto a ser traduzido e o idioma
translator_text("You need someone to hold you tight", language_destination)


# **Tradução de Documentos - Translator**

In [ ]:
# Fazer upload do documento que será traduzido
uploaded = files.upload()

In [ ]:
# Função para tradução de documentos
def translate_document(path):
  document = Document(path)
  full_text = []
  for paragraph in document.paragraphs:
    translated_text = translator_text(paragraph.text, language_destination)
    full_text.append(translated_text)

  translated_doc = Document()
  for line in full_text:
    translated_doc.add_paragraph(line)
  path_translated = path.replace('.docx', f"_{language_destination}.docx")
  translated_doc.save(path_translated)
  return path_translated

In [ ]:
  # Caminho do documento
  input_file = "/content/Tainted_Love.docx"

  # Chamada para a função de tradução de documento passando o caminho do documento como parâmetro
  translate_document(input_file)

# **Tradução de Artigos Online - Translator**

In [ ]:
!pip install requests beautifulsoup4 openai langchain-openai

In [ ]:
import requests
from bs4 import BeautifulSoup

# Função para extrair texto de uma URL
def extract_text_from_url(url):
    response = requests.get(url)

    # Verifica se a resposta da requisição a URL foi bem sucedida
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')

      # Remove as formatações de script e estilo do texto
      for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
      text = soup.get_text(separator = '')
      lines = (line.strip() for line in text.splitlines())
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      text = '\n'.join(chunk for chunk in chunks if chunk)
      return text

    # Se a resposta não for bem sucedida, exibe o código de status da resposta
    else:
        print(f"Failed to fetch the URL. Status code: {response.status_code}")
        return None

# Chamada para a função passando a URL como parâmetro
extract_text_from_url('https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo')

# **Tradução de Texto - OpenAI**

In [ ]:
# Fazer upload do arquivo .env com variáveis de ambiente da OpenAI
from google.colab import files

uploaded = files.upload()

In [ ]:
from langchain_openai.chat_models.azure import AzureChatOpenAI
from dotenv import load_dotenv
import os

# Carregar as variáveis do arquivo .env
load_dotenv('oai.env')

client = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_API_ENDPOINT"),
    api_key=os.getenv("AZURE_API_KEY"),
    api_version=os.getenv("AZURE_API_VERSION"),
    deployment_name=os.getenv("AZURE_DEPLOYMENT_NAME"),
    max_retries=0
)

def translate_article(text, lang):
  messages = [
      ("system", "Você atua como tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
  ]

  response = client.invoke(messages)
  return response.content

translate_article("Let's deploy and confirm how it works", "portugues")

'Vamos implementar e confirmar como funciona.'

# **Tradução de Artigos Online - OpenAI**

In [ ]:
url = 'https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo'
text = extract_text_from_url(url)
article = translate_article(text, 'pt-br')

print(article)